A Seq2Seq model is a model that takes a sequence of items (words, letters, time series, etc) and outputs another sequence of items.

The most common sequence-to-sequence (seq2seq)

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://drive.google.com/u/0/uc?id=19NQ87gEFYu3zOIp_VNYQZgmnwRuSIyJd&export=download
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html?ref=blog.paperspace.com

In [6]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [7]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [19]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
para-nmt-5m-processed

In [34]:
with open('para-nmt-5m-processed.txt',encoding='utf-8') as fp:
    data = fp.read()

In [38]:
a = data[:10]

In [43]:
a[0]

"so , unless that 's gon na be feasible , then ...\tso , if you can afford it , then ...\n"

In [42]:
[i.split('\t') for i in a]

[["so , unless that 's gon na be feasible , then ...",
  'so , if you can afford it , then ...\n'],
 ['of course you did .', 'of course it is .\n'],
 ["by now , singh 's probably been arrested .",
  'because he was probably just arrested at the moment by singh .\n'],
 ['not our shit . i swear .', "we did n't fuck it up , i swear .\n"],
 ['“ why not ?', '`` why not ?\n'],
 ['instead , he folded his arms and cocked his head , as if encouraging—or daring—kaye to continue .',
  'instead , he folded his hands and cocked his head as kaye encouraged - or called for - to continue .\n'],
 ["an old man 's mistake … '", "an old man 's fault ... ``\n"],
 ['he reached for a microphone and i could hear his voice booming faintly throughout the ship .',
  'he reached for the microphone , and i heard his voice faint on the ship .\n'],
 ["she said it with such simple dignity mat a lump caught in gar-ion 's throat .",
  "she said it with such simple dignity until garion 's throat tightened .\n"],
 ['he l

In [20]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [21]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['vous etes avec des amies', 'you re with friends']


In [22]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [23]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [24]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [25]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [26]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [27]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [28]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [29]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [30]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [32]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device) #AttnDecoderRNN

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
2m 1s (- 30m 24s) (5 6%) 1.7231


KeyboardInterrupt: 

In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

In [44]:
from torchtext.data import Field

ImportError: cannot import name 'Field' from 'torchtext.data' (C:\Users\avitr\anaconda3\envs\test_env\lib\site-packages\torchtext\data\__init__.py)

In [63]:
import h5py
import numpy as np
  
protein_embeddings = torch.Tensor(np.random.random((100, 512)))

file_path = 'protein_embeddings11.h5'

with h5py.File(file_path, 'w') as hf:
    hf.create_dataset('protein_embeddings', data=protein_embeddings)

In [64]:
import h5py

# Specify the path to the saved .h5 file
file_path = 'protein_embeddings11.h5'

# Open the .h5 file for reading
with h5py.File(file_path, 'r') as hf:
    # Access the dataset containing protein embeddings
    protein_embeddings_loaded = hf['protein_embeddings'][:]
    print(protein_embeddings_loaded)


[[0.9440448  0.5499231  0.61653465 ... 0.3519527  0.02355381 0.8316969 ]
 [0.59748596 0.12450572 0.86473376 ... 0.5383924  0.38188228 0.8443778 ]
 [0.9784807  0.59446365 0.52460235 ... 0.3681413  0.80170554 0.17905779]
 ...
 [0.6591164  0.3458061  0.8270602  ... 0.8642294  0.59250414 0.37902883]
 [0.04889218 0.3387536  0.3775698  ... 0.02123805 0.8473537  0.17081806]
 [0.07988291 0.39274615 0.81374127 ... 0.7998781  0.74539185 0.82511836]]


In [67]:
protein_embeddings_loaded

array([[0.9440448 , 0.5499231 , 0.61653465, ..., 0.3519527 , 0.02355381,
        0.8316969 ],
       [0.59748596, 0.12450572, 0.86473376, ..., 0.5383924 , 0.38188228,
        0.8443778 ],
       [0.9784807 , 0.59446365, 0.52460235, ..., 0.3681413 , 0.80170554,
        0.17905779],
       ...,
       [0.6591164 , 0.3458061 , 0.8270602 , ..., 0.8642294 , 0.59250414,
        0.37902883],
       [0.04889218, 0.3387536 , 0.3775698 , ..., 0.02123805, 0.8473537 ,
        0.17081806],
       [0.07988291, 0.39274615, 0.81374127, ..., 0.7998781 , 0.74539185,
        0.82511836]], dtype=float32)

In [48]:
type(f2)

h5py._hl.files.File

In [49]:
l = list(f2.keys() )
print(len(l), l[:10])

1 ['protein_embeddings']


In [53]:
f2['protein_embeddings'][:20]

array([[0.77292372, 0.6362704 , 0.86815597, ..., 0.29162078, 0.29937216,
        0.56395325],
       [0.56311823, 0.7305264 , 0.44362651, ..., 0.70128231, 0.35462791,
        0.01370367],
       [0.44822637, 0.96026877, 0.92195717, ..., 0.21605968, 0.5387826 ,
        0.65719921],
       ...,
       [0.64517707, 0.81052868, 0.20803794, ..., 0.56407871, 0.69050443,
        0.34868391],
       [0.91223094, 0.97778511, 0.99379934, ..., 0.04774938, 0.26620207,
        0.35191414],
       [0.33951937, 0.34531123, 0.01255089, ..., 0.06614991, 0.28559942,
        0.33451955]])

In [55]:
protein_embeddings.shape

(100, 512)

In [56]:
import torch

tensor([[0.7729, 0.6363, 0.8682,  ..., 0.2916, 0.2994, 0.5640],
        [0.5631, 0.7305, 0.4436,  ..., 0.7013, 0.3546, 0.0137],
        [0.4482, 0.9603, 0.9220,  ..., 0.2161, 0.5388, 0.6572],
        ...,
        [0.0280, 0.4254, 0.2756,  ..., 0.0989, 0.7402, 0.7849],
        [0.0686, 0.0517, 0.7099,  ..., 0.7014, 0.6886, 0.3220],
        [0.5068, 0.5967, 0.5074,  ..., 0.2945, 0.9491, 0.3344]])